In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
compute_cluster = ComputeTarget.create(ws, "whitehat", config)
compute_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--max_iter': choice(200,300, 400, 50)
    '--C': choice(0.001, 0.005, 0.01, 0.05)
}

)

# Specify a Policy
policy = BanditPolicy(slack_amount = 0.2, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="training",
                          pip_packages=['azureml-sdk'],
                          entry_script='train.py',
                          compute_target = compute_cluster,)
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name='AUC', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6,
                          max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

experiment = Experiment(workspace = ws, name = 'diabates_training_hyperdrive')
run = experiment.submit(config=hyperdrive)
RunDetails(run).show()
run.wait_for_completion()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run_hyperdrive = run.get_best_run_by_primary_metric()
best_run_hyperdrive.register_model(model_path='outputs/model.pkl', model_name='Best_model_Hiper_drive')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory as tdf

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
### YOUR CODE HERE ###
ds = tdf.from_delimited_files(data_url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)
X_Train = pd.concat([x_train, y_train], axis=1, sort=False)
X_test = pd.concat([x_test, y_test], axis=1, sort=False)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    compute_target=compute_cluster,
    training_data=X_Train
    validation_data = X_test
    label_column_name='y',
    n_cross_validations= 3
    featurization='auto',)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_experiment = Experiment(ws, 'Banking_data_automl')
automl_run = automl_experiment.submit(automl_config)
automl_run.wait_for_completion(show_output=True)
print("Model Trained using AutoML !!")

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_automl, fitted_model = automl_run.get_output()

best_run_automl.register_model(model_path='outputs/model.pkl', model_name='Best_model_automl')

In [ ]:
## Deleting the compute cluster created !!
delete_compute(compute_cluster)